In [62]:
import numpy as np
import pandas as pd
import datetime
import sys, os

# Data parsing

In [63]:
# Parse html file
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 20
# MONTH = 5
# SUB = '/scrf'
SUB = ''
file_name = 'Оперативные данные _ Коронавирус COVID–19_ Официальная информация о коронавирусе в России на портале – стопкоронавирус.рф.html'

# Save of "https://xn--80aesfpebagmfblc0a.xn--p1ai/information/" page
path = f'html_data/{DAY}-{MONTH}{SUB}/{file_name}'
parse_df = pd.read_html(path, encoding='utf-8')[0]

parse_df.head()

,Регион,Выявлено,Новые,Активные ? Число больных в настоящее время,Выздоровело,Умерло
0,Москва,180791,2595,98135,80179,2477
1,Московская область,38995,757,30455,8091,449
2,Санкт-Петербург,15949,369,10065,5680,204
3,Нижегородская область,9834,301,6043,3693,98
4,Свердловская область,5449,265,3249,2173,27


In [64]:
# Modifi data frame
parse_df = parse_df.drop(parse_df.columns[2:4] , axis=1)
parse_df.columns = ['Region/City', 'Confirmed', 'Recovered', 'Deaths']
parse_df['Date'] = datetime.datetime.now().strftime("%Y-%m-%d")
parse_df

,Region/City,Confirmed,Recovered,Deaths,Date
0,Москва,180791,80179,2477,2020-05-31
1,Московская область,38995,8091,449,2020-05-31
2,Санкт-Петербург,15949,5680,204,2020-05-31
3,Нижегородская область,9834,3693,98,2020-05-31
4,Свердловская область,5449,2173,27,2020-05-31
...,...,...,...,...,...
80,Севастополь,161,131,3,2020-05-31
81,Республика Алтай,122,71,0,2020-05-31
82,Сахалинская область,115,53,0,2020-05-31
83,Чукотский автономный округ,80,54,1,2020-05-31


# Preparing parsed data

In [65]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
6815,2020-05-30,Челябинская область,Chelyabinsk region,74.0,150.0,1.0,74.0,2659.0,16.0,1180.0
6816,2020-05-30,Чеченская Республика,Chechen Republic,95.0,17.0,0.0,38.0,1209.0,13.0,825.0
6817,2020-05-30,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,5.0,0.0,3.0,78.0,1.0,53.0
6818,2020-05-30,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,30.0,0.0,55.0,2310.0,9.0,953.0
6819,2020-05-30,Ярославская область,Yaroslavl region,76.0,79.0,0.0,53.0,3055.0,14.0,751.0


In [66]:
# Add day columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

# Rename regions
rename_dict = {
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
}

parse_df['Region/City'] = parse_df['Region/City'].replace(rename_dict)

In [67]:
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# Minus a day if needed
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [68]:
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Day-Confirmed'].sum()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Day-Deaths'].sum()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Day-Recovered'].sum()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [69]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [70]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 405843 Day-Confirmed: 9268.0
Russia Deaths: 4693 Day-Deaths: 138.0
Russia Recovered: 171883 Day-Recovered: 4414.0


# Data saving

In [71]:
# Save full update table
parse_df.to_csv(f'covid19-russia-cases-upd-full-scrf.csv', index=False)

In [72]:
# Drop rows without day changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [73]:
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}-scrf.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd-scrf.csv', index=False)